In [2]:
import pandas as pd
import numpy as np

In [1]:
def CV_process(input_file, cycle_num, electrode_area):
    names = ['Potential (V vs Ag/AgCl)', 'Current (mA)']
    df = pd.read_csv(input_file, sep = '\t', index_col = False, names = names)

    str_row_num = 0
    index_list = df.index.tolist()
    for index in index_list:
        try:
            float(df.loc[index, names[0]])
            break
        except ValueError:
            str_row_num += 1
    if str_row_num == 0:
        df = pd.read_csv(input_file, sep = '\t', names = names)
    else:
        df = pd.read_csv(input_file, sep = '\t', header = str_row_num-1, index_col = False, names = names)
        
    count = 0
    index_list = df.index.tolist()
    for index in index_list:
        if df.loc[index, names[0]] < df.loc[index + 1, names[0]]:
            count += 1
        else:
            break
    starting_index = (cycle_num - 1) * 2 * count
    # actually I should add 2 in the edn to count the two non-integer line, but when I do plot, I found that the circle is not complete,
    # so I removed the 2 to make the circle complete.
    df_cropped = pd.read_csv(input_file, sep = '\t', header = starting_index, index_col = False, names = names, dtype = {names[0]: float, names[1]: float})
    df_cropped['Current density ($mA/cm^2$)'] = df_cropped[names[1]] / electrode_area
    return df_cropped

In [2]:
#aim to keep only the last CP cycle

def CP_process(input_file, current_density, mass_loading):
    names = ['Time (s)', 'Potential (V vs Ag/AgCl)']
    df = pd.read_csv(input_file, sep = '\t', names = names)

    str_row_num = 0
    index_list = df.index.tolist()
    for index in index_list:
        try:
            float(df.loc[index, names[0]])
            break
        except ValueError:
            str_row_num += 1
    if str_row_num == 0:
        df = pd.read_csv(input_file, sep = '\t', index_col = False, names = names, dtype = {names[0]: float, names[1]: float})
    else:
        df = pd.read_csv(input_file, sep = '\t', header = str_row_num-1, index_col = False, names = names, dtype = {names[0]: float, names[1]: float})

        
    def loop_interval(df, intervals, shorter_index_list):
        turning_point = []
        for interval in intervals:
            for index in shorter_index_list:
                count = 1
                for i in range(1, interval):
                    if (df.loc[index-i, names[1]] < df.loc[index, names[1]]) and (df.loc[index, names[1]] > df.loc[index+i, names[1]]):
                        count += 1
                if count==interval:
                    turning_point.append(index)
            if len(turning_point)==2:
                return turning_point
            else:
                shorter_index_list = turning_point
                turning_point = []
        return turning_point


    index_list = df.index.tolist()
    if len(index_list)>100:
        upper_limit = len(index_list) // 100
        shorter_index_list = index_list[upper_limit:(len(index_list)-upper_limit)]
        exp_index = int(np.log2(upper_limit))
        intervals = np.geomspace(2, 2**exp_index, exp_index, dtype = int)
        turning_point = loop_interval(df, intervals, shorter_index_list)
    else:
        shorter_index_list = index_list[16:(len(index_list)-16)]
        intervals = [2, 4, 8, 16]
        turning_point = loop_interval(df, intervals, shorter_index_list)

    starting_index = max(turning_point)
#     index_list = df.index.tolist()
#     upper_limit = len(index_list) // 20
#     shorter_index_list = index_list[upper_limit:(len(index_list)-upper_limit)]
#     turning_point = []
#     increase = max([10, (upper_limit-5)//20])
#     intervals = range(5, upper_limit, increase)
#     for interval in intervals:
#         for index in shorter_index_list:
#             count = 1
#             for i in range(1, interval):
#                 if (df.loc[index-i, names[1]] < df.loc[index, names[1]]) and (df.loc[index, names[1]] > df.loc[index+i, names[1]]):
#                     count += 1
#             if count==interval:
#                 turning_point.append(index)
#         if len(turning_point)==2:
#             break
#         else:
#             shorter_index_list = turning_point
#             turning_point = []

#     starting_index = max(turning_point)

    time_list = df[names[0]].tolist()
    cropped = pd.read_csv(input_file, sep = '\t', header = starting_index+1, index_col = False, names = names, dtype = {names[0]: float, names[1]: float})

    cropped.loc[:, names[0]] = cropped.loc[:, names[0]] - time_list[starting_index]
    cropped[names[1]] = -cropped[names[1]]
    cropped.reset_index(drop = True, inplace = True)
    

# this piece of code is calculating iR_drop
    max_index = cropped[names[1]].idxmax()
    iR_drop_list = []
    for i in range(20):
        if cropped.loc[max_index+i, names[1]] > cropped.loc[max_index+i+1, names[1]]:
            iR_drop_list.append(cropped.loc[max_index+i, names[1]] - cropped.loc[max_index+i+1, names[1]])
    iR_drop = max(iR_drop_list)
    
# this piece of code is calculating the supercapacitance
    t1 = cropped.loc[max_index, names[0]]
    time_list = cropped[names[0]].tolist()
    t2 = time_list[-1]
    areal_C = current_density * 0.001 * (t2 - t1) / (0.8 - iR_drop)
    gra_C = areal_C / mass_loading # 1 F/cm2 / 1 mg/cm2 = 1F/mg
    
    return cropped, iR_drop, areal_C, gra_C

In [5]:
def find_Rs_and_fitting(list1, list2):
    coef = np.polyfit(list1, list2, 1)
    poly1d_fn = np.poly1d(coef)
    Rs = coef[0] * 1000
    print(f'Rs = {Rs} ohm')
    return poly1d_fn

In [6]:
def retention(input_list):
    retention_pec = input_list / input_list[0] * 100
    retention_pec = retention_pec.tolist()
    return retention_pec

In [7]:
def EIS(input_file):
    names = ['Re (ohm)', 'Im (ohm)']
    df = pd.read_csv(input_file, sep = '\t', index_col = False, names = names)
    str_row_num = 0
    index_list = df.index.tolist()
    for index in index_list:
        try:
            float(df.loc[index, names[0]])
            break
        except ValueError:
            str_row_num += 1
    if str_row_num == 0:
        df = pd.read_csv(input_file, sep = '\t', index_col = False, names = names)
    else:
        df = pd.read_csv(input_file, sep = '\t', header = str_row_num-1, index_col = False, names = names)
    return df